# 資料合併-長程

In [ ]:
import os
import pandas as pd

def merge_files(file_paths, output_path):
    data_frames = []
    
    # 逐一讀取檔案
    for file_path in file_paths:
        if os.path.exists(file_path):
            print(f"正在讀取檔案：{file_path}")
            df = pd.read_csv(file_path)
            if not df.empty:
                data_frames.append(df)
            else:
                print(f"檔案 {file_path} 為空，跳過。")
        else:
            print(f"檔案 {file_path} 不存在，跳過。")

    # 合併所有資料
    if data_frames:
        merged_data = pd.concat(data_frames, ignore_index=True)
        merged_data.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"合併完成，結果儲存至：{output_path}")
    else:
        print("沒有任何資料可供合併。")

# 主程式
base_path = '/Users/yuchingchen/Documents/專題/cleaned_data/data/long'
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight.csv'

# 檔案名稱清單
file_paths = [
    f'{base_path}/sydney.csv',
    f'{base_path}/zurich.csv',
    f'{base_path}/losangeles.csv',
    f'{base_path}/frankfurt.csv',
    f'{base_path}/france.csv',
    f'{base_path}/newyork.csv',
    f'{base_path}/london.csv',
]

# 合併檔案
merge_files(file_paths, output_path)

# 資料合併-長程-商務艙

In [ ]:
import os
import pandas as pd

def merge_files(file_paths, output_path):
    data_frames = []
    
    # 逐一讀取檔案
    for file_path in file_paths:
        if os.path.exists(file_path):
            print(f"正在讀取檔案：{file_path}")
            df = pd.read_csv(file_path)
            if not df.empty:
                data_frames.append(df)
            else:
                print(f"檔案 {file_path} 為空，跳過。")
        else:
            print(f"檔案 {file_path} 不存在，跳過。")

    # 合併所有資料
    if data_frames:
        merged_data = pd.concat(data_frames, ignore_index=True)
        merged_data.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"合併完成，結果儲存至：{output_path}")
    else:
        print("沒有任何資料可供合併。")

# 主程式
base_path = '/Users/yuchingchen/Documents/專題/cleaned_data/data/long'
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight_bsuiness.csv'

# 檔案名稱清單
file_paths = [
    f'{base_path}/sydney_business.csv',
    f'{base_path}/zurich_business.csv',
    f'{base_path}/losangeles_business.csv',
    f'{base_path}/frankfurt_business.csv',
    f'{base_path}/france_business.csv',
    f'{base_path}/newyork_business.csv',
    f'{base_path}/london_business.csv',
]

# 合併檔案
merge_files(file_paths, output_path)

# 長程航班-經濟艙

In [9]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 載入資料並刪除價格歷史為空的列
data = pd.read_csv('/Users/yuchingchen/Documents/專題/cleaned_data/long_flight.csv')

# 統一日期格式為 "YYYY-MM-DD"
data["出發日期"] = pd.to_datetime(
    data["出發日期"], 
    format="%Y-%m-%d", 
    errors="coerce"
).combine_first(
    pd.to_datetime(data["出發日期"], format="%Y/%m/%d", errors="coerce")
)
data["出發日期"] = data["出發日期"].dt.strftime("%Y-%m-%d")

# 計算對數變數
data["平均價格_log"] = data["平均價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["最低價格_log"] = data["最低價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["價格變異_log"] = data["價格變異"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["中位數價格_log"] = data["中位數價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)

# 91 - '最低價格剩餘天數' 
data['最低價格天數'] = 91 - data['最低價格剩餘天數']

# 移除包含 NaN 的數據
data = data.dropna(subset=["平均價格_log", "最低價格_log", "最低價格天數", "價格變異_log", "中位數價格_log"])

# 顯示抵達機場代號種類及數量
print(data['抵達機場代號'].value_counts())

/var/folders/ty/th__2_pn1js9hzn2y7l830wr0000gn/T/ipykernel_92531/4292337466.py:8: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/yuchingchen/Documents/專題/cleaned_data/long_flight.csv')


抵達機場代號
LHR    11504
LAX     9489
FRA     8405
SYD     7867
CDG     6727
JFK     5993
ZRH     5691
Name: count, dtype: int64


In [15]:
# 定義假期範圍
holidays = {
    "台灣": [("2025-01-25", "2025-02-02"), ("2025-02-28", "2025-03-02")],
    "美國": [("2025-01-18", "2025-01-20"), ("2025-02-15", "2025-02-17")],
    "澳洲": [("2025-01-25", "2025-01-27")],
    "瑞士": [("2025-01-29", "2025-01-30")]
}

# 抵達機場代號與地區的對應關係
airport_to_region = {
    "LAX": "美國", # 洛杉磯
    "SYD": "澳洲", # 雪梨
    "FRA": "德國", # 法蘭克福
    "CDG": "法國", # 巴黎
    "JFK": "美國", # 紐約
    "LHR": "英國", # 倫敦
    "ZRH": "瑞士"  # 蘇黎世
}

# 轉換日期類型
data["出發日期"] = pd.to_datetime(data["出發日期"])

# 定義一個函數來檢查是否為假期
def is_holiday(row):
    # 台灣的假期（不管去哪，出發地是台灣）
    for start, end in holidays["台灣"]:
        if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
            return 1  # 假期
    # 其他地區的假期
    region = airport_to_region.get(row["抵達機場代號"])
    if not region or region not in holidays:
        return 0  # 沒有假期
    for start, end in holidays[region]:
        if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
            return 1  # 假期
    return 0  # 非假期

# 新增假期欄位
data["假期"] = data.apply(is_holiday, axis=1)

# 調整「假期」到「艙等」後面
cols = data.columns.tolist()
if '假期' in cols and '艙等' in cols:
    cols.insert(cols.index('艙等') + 1, cols.pop(cols.index('假期')))
data = data[cols]

# 新增 "Cost of Living Index" 和 "GDP (PPP) per capita (in thousand USD)" 資料
additional_data = {
    "Region": ["美國", "澳洲", "德國", "英國", "法國", "瑞士"],
    "Cost of Living Index": [70.4, 70.2, 62.2, 62.0, 63.7, 101.1],
    "GDP (PPP) per capita (in thousand USD)": [82.715, 67.901, 69.532, 60.735, 63.881, 93.055],
}

# 將 Region 對應到現有的抵達機場代號的地區
region_df = pd.DataFrame(additional_data)

# 透過 `airport_to_region` 映射，新增這兩個欄位到原始資料
data["Region"] = data["抵達機場代號"].map(airport_to_region)
data = data.merge(region_df, how="left", on="Region")

# 匯出新的資料集
data.to_csv('/Users/yuchingchen/Documents/專題/cleaned_data/long_flight_final.csv', index=False)

In [21]:
# 顯示機型種類及數量
print(data['機型分類'].value_counts())

# 顯示 機型分類 = -1 的資料
print(data[data['機型分類'] == -1])

機型分類
 1    4225
 0    2770
 2      61
-1      52
Name: count, dtype: int64
           出發日期  星期     出發時間  出發時段 出發機場代號     抵達時間  抵達時段 抵達機場代號  航空公司  航空聯盟  \
6232 2025-01-21  週二   下午2:30  下午班機    TSA   下午6:00  晚間抵達    GMP  長榮航空     1   
6241 2025-01-21  週二   清晨7:05  早晨班機    TPE  上午10:30  上午抵達    ICN  長榮航空     1   
6247 2025-01-22  週三   下午3:45  下午班機    TSA   晚上7:15  晚間抵達    GMP  長榮航空     1   
6268 2025-01-23  週四   清晨7:05  早晨班機    TPE  上午10:30  上午抵達    ICN  長榮航空     1   
6271 2025-01-24  週五   上午9:20  上午班機    TSA  中午12:50  下午抵達    GMP  長榮航空     1   
6285 2025-01-25  週六   上午9:20  上午班機    TSA  中午12:50  下午抵達    GMP  長榮航空     1   
6297 2025-01-25  週六   清晨7:05  早晨班機    TPE  上午10:30  上午抵達    ICN  長榮航空     1   
6334 2025-01-28  週二   下午2:30  下午班機    TSA   下午6:00  晚間抵達    GMP  長榮航空     1   
6345 2025-01-28  週二   清晨7:05  早晨班機    TPE  上午10:30  上午抵達    ICN  長榮航空     1   
6351 2025-01-29  週三   下午3:45  下午班機    TSA   晚上7:15  晚間抵達    GMP  長榮航空     1   
6356 2025-01-29  週三  中午12:40  下午班機    TPE   下午4:05  下午抵達